In [1]:
import re
import os
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json
import subprocess
from shutil import copytree, ignore_patterns

In [ ]:
from core.compilers import CompilationStatus
from core.compilers import drizzle as drizzle_compiler
from core.compilers import typespec as typespec_compiler

In [3]:
class Builder:
    def __init__(self, client: AnthropicBedrock, template_dir: str = "templates"):
        self.client = client
        self.template_dir = template_dir
        self.jinja_env = jinja2.Environment()
        self.typespec_tpl = self.jinja_env.from_string(stages.typespec.PROMPT)
        self.drizzle_tpl = self.jinja_env.from_string(stages.drizzle.PROMPT)
        self.router_tpl = self.jinja_env.from_string(stages.router.PROMPT)
        self._model = "anthropic.claude-3-5-sonnet-20241022-v2:0"

    def build(self, application_description: str, out_dir: str = "app_output"):
        copytree(self.template_dir, out_dir, ignore=ignore_patterns('*.pyc', '__pycache__')) # 'node_modules'
        compiler_typespec = typespec_compiler.TypeSpecCompiler(out_dir)
        prompt_typespec = self.typespec_tpl.render(
            application_description=application_description,
        )
        typespec_response = self.client.messages.create(
            model=self._model,
            max_tokens=8192,
            messages=[{"role": "user", "content": prompt_typespec}]
        )
        typespec_definitions = stages.typespec.parse_output(typespec_response.content[0].text)
        typespec_result = compiler_typespec.compile(typespec_definitions["typespec_definitions"])
        print("finished typespec")
        #if typespec_result["result"] != CompilationStatus.SUCCESS:
        #    raise Exception("Failed to compile typespec")
        
        compiler_drizzle = drizzle_compiler.DrizzleCompiler(out_dir)
        prompt_drizzle = self.drizzle_tpl.render(
            typespec_definitions=typespec_definitions["typespec_definitions"],
        )
        drizzle_response = self.client.messages.create(
            model=self._model,
            max_tokens=8192,
            messages=[{"role": "user", "content": prompt_drizzle}]
        )
        drizzle_schema = stages.drizzle.parse_output(drizzle_response.content[0].text)
        drizzle_result = compiler_drizzle.compile(drizzle_schema["drizzle_schema"])
        print("finished drizzle")
        #if drizzle_result["result"] != CompilationStatus.SUCCESS:
        #    raise Exception("Failed to compile drizzle")
        
        prompt_router = self.router_tpl.render(
            typespec_definitions=typespec_definitions["typespec_definitions"],
            user_request=application_description,
        )
        router_response = self.client.messages.create(
            model=self._model,
            max_tokens=8192,
            messages=[{"role": "user", "content": prompt_router}]
        )
        print("ROUTER_RESPONSE", router_response)
        router_result = stages.router.parse_outputs([
            content for content in router_response.content
        ])["user_functions"]
        return {
            "typespec": typespec_result,
            "drizzle": drizzle_result,
            "router": router_result
        }

In [4]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [5]:
builder = Builder(client, template_dir="templates")

In [6]:
app_result = builder.build(
    "Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines for specific list of available equipment and time constraints.",
    out_dir="app_output"
)

finished typespec
start checking drizzle
end checking drizzle
finished drizzle
ROUTER_RESPONSE Message(id='msg_bdrk_01Y7vDHFbAWdgfcGA12Epx74', content=[TextBlock(text='I\'ll help create a classification prompt for the GymTracker interface functions.\n\nHere\'s the structured function classification guide:\n\n{\n  "user_functions": [\n    {\n      "name": "recordExercise",\n      "description": "Records a completed exercise with details like sets, reps, weight, and duration. Use this when the user wants to log or track an exercise they\'ve performed.",\n      "examples": [\n        "I just did 3 sets of 12 bench presses with 135 pounds",\n        "Completed 4 sets of squats, 8 reps each at 185 lbs",\n        "Did 30 minutes on the treadmill",\n        "Logged my deadlift workout: 5 sets of 5 reps at 225 pounds"\n      ]\n    },\n    {\n      "name": "getProgress",\n      "description": "Retrieves and analyzes progress for a specific exercise over time. Use this when users want to check 

TypeError: 'NoneType' object is not subscriptable

In [ ]:
app_result